In [ ]:
import random
import time

In [ ]:
import cloudscraper
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Buffalo Wild Wings Data

In [6]:
scraper = cloudscraper.create_scraper()

In [7]:
store_lists = []
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    state_code = states_df.iloc[i]["STUSPS"]
    url = f"https://api-idp.buffalowildwings.com/bww/web-exp-api/v1/location/list/details?countryCode=US&stateOrProvinceCode={state_code}"
    r = scraper.get(url)

    if r.status_code != 200:
        print(state_code)
        continue

    json_data = r.json()
    for city in json_data["stateOrProvince"]["cities"]:
        for location in city["locations"]:
            point = Point(
                location["details"]["longitude"], location["details"]["latitude"]
            )
            location["geometry"] = point
            location["STUSPS"] = location["contactDetails"]["address"][
                "stateProvinceCode"
            ]
            store_lists.append(location)

    time.sleep(random.uniform(0.01, 0.5))

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

PR
AS
VI
GU
MP


In [8]:
bww_gdf = gpd.GeoDataFrame(store_lists, crs=4326)

In [15]:
bww_gdf = bww_gdf.to_crs(9311)
bww_gdf.to_file(f"data/bww.gpkg")

## Get Population Data

In [16]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [17]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [18]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [19]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [21]:
bww_state_counts_df = pd.DataFrame(
    bww_gdf.groupby("STUSPS").size(), columns=["BWWs"]
).reset_index()

In [22]:
bww_state_counts_gdf = states_with_population_df.merge(
    bww_state_counts_df, on="STUSPS", how="left"
)
bww_state_counts_gdf = bww_state_counts_gdf.fillna(0)

In [23]:
bww_state_counts_gdf["per_100k"] = bww_state_counts_gdf["BWWs"] / (
    bww_state_counts_gdf["POPULATION"] / 100000
)
bww_state_counts_gdf["per_1m"] = bww_state_counts_gdf["BWWs"] / (
    bww_state_counts_gdf["POPULATION"] / 1_000_000
)

In [24]:
bww_state_counts_gdf = bww_state_counts_gdf.to_crs(9311)

In [25]:
bww_state_counts_gdf.to_file("data/bww_per_state.gpkg")